## Spotify  tabular  data  collection

In [ ]:
import pandas as pd
import ast
import json
import os
import sys
import spotipy
import spotipy.util as util
import requests
from spotipy import oauth2

In [ ]:

def infos(content = False):   
    
    if content == False:
        global client_id
        global client_secret
        global redirect_uri
        global username
        global token
        global user_id
        global scope
        
        client_id='b8ff3d65f36e4406a1f297536869ed25'
        user_id='70b2fxjh391lur5jysab53kzw' 
        client_secret='43a0f4024df2424d984893f86801cb92'
        redirect_uri='http://localhost:8018/callback'
        username = 'Lucas_Paiva'
        scope = 'playlist-modify-public user-read-email'
        #token = 'BQA6zuo3icYN5xg-TWVJJXm0ePbCcHw6crAyttjiLPI5FDA-u2yacYDU6NTziCHaUL_ncyAUSob9X8WE0rE9U2tHLbnO8G447cMLKpfv9OGmXeJ7Qo4jrR61zHWoc6fHrDtVQ9zk3GEMhMmDZ8OwJXihnYHTFove6CY9G-YuO9i3gNskibVyxPIDppz247vShPIdTFSf7okWIoVKx_l2imFmPaUYR40Lhz2G-v7GaGRtCy5V'


        token = util.prompt_for_user_token(username,
                                            scope,
                                            client_id=client_id,
                                            client_secret=client_secret,
                                            redirect_uri=redirect_uri)
                                       
    print('Informações: client_id, user_id, client_secret, redirect_uri, username, scope, token')
    
# infos()


### Search from keywords 

In [ ]:
def get_musicas (chaves,offset=1000):
# artista = genero

    infos()
    tracks = []
    result =[]
    numerochaves = len(chaves)

    if token:
        sp = spotipy.Spotify(auth=token)
        sp.trace = False
        
        
        result = sp.search(chaves[0],market='BR',limit=1)
        dat = pd.DataFrame(result['tracks']['items']).copy()
        for x in range(0, numerochaves):
            print(chaves[x])
        
            for off in range (0,offset,50):
                result = sp.search(chaves[x],market='BR',limit=50,offset=off) 
                
                for i, t in enumerate(result['tracks']['items']):
                    tracks.append(str(t['id'].strip( 'u' )))
                    
                result = pd.DataFrame(result['tracks']['items']).copy()
                data = pd.concat([dat,result]).copy()
                dat=data.copy()
            
            print('dat',dat.shape)
               
    
    return dat



In [ ]:
chaves= ['Falamansa','Rastapé',
        'Circuladô de Fulô','Estakazero','Trio Virgulino','Trio Nordestino','Forróçacana',
        'Bicho de Pé', 'Trio Forrozão','Forrueiros','Raiz do Sana','Chama Chuva',
             
        'Luiz Gonzaga', 'Rastapé','Trio Dona Zefa','Severino Januário','Dominguinhos',
        'Santanna "O cantador"','Trio Pé De Serra',  'Pe de Serra Forro Band',
        
        'Forró Universitário', 'Forró Pé de Serra','Forró Tradicional', 'Xote', 'Baião'
        ]

teste=['Trio Virgulino','Forróçacana']

data_info = get_musicas()

### Data cleaning

In [ ]:
def filtro_base(data_info):
    
    data_info = data_info.drop_duplicates(subset=['id']) #duplicates ID
    data_info = data_info.drop_duplicates(subset=['name']) #duplicates Name
    data_info = data_info[data_info['explicit']!=True] #remove explicit
    data_info = data_info[data_info['is_playable']==True] #remove no_playable
    data_info = data_info.dropna()               #remove NAN
    
    data_info = data_info.reset_index(drop=True)

    return data_info



In [ ]:
data_info = filtro_base(data_info)

### Getting  GI,  AF  and  AA

#### General information data

In [ ]:
def more_info(data_info):
    infos()
    artistas = data_info.artists.values
    nome_artista = []
    id_artista = []
    
    for i in range(0,len(artistas)):
        nome_artista.append(artistas[i][0]['name'])
        id_artista.append(artistas[i][0]['id'])

    # print(nome_artista,id_artista)

    data_info['nome_artista'] = nome_artista
    data_info['id_artista'] = id_artista


    #Inserindo Informações dos albuns
    albuns = data_info.album.values
    nome_album = []
    id_album = []

    for i in range(0,len(albuns)):
        nome_album.append(albuns[i]['name'])
        id_album.append(albuns[i]['id'])

    # print(nome_artista,id_artista)

    data_info['nome_album'] = nome_album
    data_info['id_album'] = id_album
    
    return data_info

In [ ]:
data_inicio = more_info(data_info)
print(data_inicio.shape)
data_inicio = data_inicio.rename(columns={'album':'album_data',"id":"track_id",
                "nome_artista":'artist',"id_artista":"artist_id", 'nome_album':'album'
                ,"id_album": "album_id", "name": "track"})
data_inicio.head()

In [ ]:
year = []
for j in range (0,data_inicio.shape[0]):
    
    valor = data_inicio['album_data'].values[j]
    valor= ast.literal_eval(valor)
    year.append(int(valor['release_date'][0:4]))
data_inicio['track_year'] = year

In [ ]:
data_GI = data_inicio[['track','track_id',  'artist', 'artist_id','popularity', 'album','album_id', 'track_year', 'duration_ms',
                           'uri','preview_url']]
data_GI.head()
data_GI.shape

In [ ]:
data_GI.head()

In [ ]:
data_GI.to_csv('general_information_data.csv')

#### Audio features data

In [ ]:
def get_features(data_play):

    infos()

    offset = 0
    songs = []
    items = []

    sp = spotipy.Spotify(auth=token)
    ids = list(data_play.track_id)

    index = 0
    audio_features = []
    
 
    while index < len(ids):
        audio_features += sp.audio_features(ids[index:index + 50])
        index += 50

    features_list = []
    i=0
    
    for features in audio_features:
        features_list.append([features['id'],features['energy'], features['liveness'],
                              features['tempo'], features['speechiness'],
                              features['acousticness'], features['instrumentalness'],
                              features['time_signature'], features['danceability'],
                              features['key'], features['loudness'], features['valence'],
                              features['mode']])
    
    df = pd.DataFrame(features_list, columns=['track_id','energy', 'liveness',
                                              'tempo', 'speechiness',
                                              'acousticness', 'instrumentalness',
                                              'time_signature', 'danceability',
                                              'key', 'loudness',
                                              'valence', 'mode'])
    return df

In [ ]:
data_GI = pd.read_csv('general_information_data.csv',index_col=0)

In [ ]:
data_AF = get_features(data_GI)

In [ ]:
data_AF.to_csv('audio_features_data.csv')

#### Audio analysis data

In [ ]:
def get_analises(data_play):

    infos()
    
    ids = data_play.track_id
    sp = spotipy.Spotify(auth=token)

    index = 0

    analises = []
    analises_list = []
    
    print(len(ids))
    
    while index <len(ids):
        analises = sp.audio_analysis(ids[index])
        analises_list.append([ids[index],analises['bars'],analises['beats'],analises['tatums']])
        index +=1
        if index%100 ==0:
            infos()
            sp = spotipy.Spotify(auth=token)
            print(index)
    
    df = pd.DataFrame(analises_list, columns=['track_id','bars','beats','tatums'])#,'bars_anot','beats_anot','tatums_anot'
    return df

In [ ]:
data_GI = pd.read_csv('general_information_data.csv',index_col=0)

In [ ]:
data_AA = get_analises(data_GI)

In [ ]:
data_AA.to_csv('audio_analysis_data.csv')

#### Merging all Spotify data

In [ ]:
data_GI_AF = data_GI.merge(data_AF, how='left',on='track_id')
data_GI_AF_AA = data_GI_AF .merge(data_AA, how='left',on='track_id')

In [ ]:
data_GI_AF_AA.columns

In [ ]:
data_GI_AF_AA .to_csv('spotify_tabular_data.csv')